In [1]:
import numpy as np
import xgboost as xgb
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score
# Import libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from time import time
import matplotlib.pyplot as plt
from operator import itemgetter

# Stop deprecation warnings from being printed
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
!ls 

datalab


In [1]:
!wget https://s3.ap-south-1.amazonaws.com/venkata512/train_new_feature.csv

--2018-08-13 14:25:14--  https://s3.ap-south-1.amazonaws.com/venkata512/train_new_feature.csv
Resolving s3.ap-south-1.amazonaws.com (s3.ap-south-1.amazonaws.com)... 52.219.62.5
Connecting to s3.ap-south-1.amazonaws.com (s3.ap-south-1.amazonaws.com)|52.219.62.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138510298 (132M) [text/csv]
Saving to: ‘train_new_feature.csv’

train_new_feature.c 100%[===================>] 132.09M  4.31MB/s    in 75s     

2018-08-13 14:26:30 (1.77 MB/s) - ‘train_new_feature.csv’ saved [138510298/138510298]



In [0]:
train = pd.read_csv("train_new_feature.csv")
#test = pd.read_csv("test_building.csv")

In [0]:
columns=train.columns

In [23]:
columns

Index(['area_assesed', 'damage_grade', 'has_geotechnical_risk',
       'has_repair_started', 'legal_ownership_status', 'count_families',
       'has_secondary_use', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'condition_post_eq', 'gelogical', 'secondary',
       'superstructure'],
      dtype='object')

In [0]:
categorical_features=['area_assesed','has_repair_started', 'legal_ownership_status',
       'land_surface_condition','foundation_type','roof_type', 'ground_floor_type', 'other_floor_type',
       'position', 'plan_configuration', 'condition_post_eq','has_geotechnical_risk','has_secondary_use']

In [14]:
categorical_features

['area_assesed',
 'has_repair_started',
 'legal_ownership_status',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'condition_post_eq',
 'has_geotechnical_risk',
 'has_secondary_use']

In [0]:
numerical_features=['count_families',
       'count_floors_pre_eq', 'count_floors_post_eq', 'age_building',
       'plinth_area_sq_ft', 'height_ft_pre_eq', 'height_ft_post_eq',
       'gelogical', 'secondary', 'superstructure']

In [0]:
train.fillna('-999.0', inplace=True)

In [0]:
numerical_features=train.drop(categorical_features, axis=1, inplace=False).columns

In [0]:
train.drop('Unnamed: 0', axis=1, inplace=True)
train.drop('building_id', axis=1, inplace=True)

#test.drop('Unnamed: 0', axis=1, inplace=True)

In [0]:
train[categorical_features]=train[categorical_features].apply(lambda x: x.astype('category'))

In [0]:
train[numerical_features]=train[numerical_features].apply(lambda x: x.astype('float'))

In [27]:
train.dtypes

area_assesed              category
damage_grade                 int64
has_geotechnical_risk     category
has_repair_started        category
legal_ownership_status    category
count_families             float64
has_secondary_use         category
count_floors_pre_eq          int64
count_floors_post_eq         int64
age_building                 int64
plinth_area_sq_ft            int64
height_ft_pre_eq             int64
height_ft_post_eq            int64
land_surface_condition    category
foundation_type           category
roof_type                 category
ground_floor_type         category
other_floor_type          category
position                  category
plan_configuration        category
condition_post_eq         category
gelogical                    int64
secondary                    int64
superstructure               int64
dtype: object

In [0]:
train_new=pd.get_dummies(train, columns=categorical_features)

In [13]:
test_new=pd.get_dummies(test, columns=categorical_features)

NameError: ignored

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_new, train_new.damage_grade, test_size=0.35, random_state=42)

In [0]:
cols = [col for col in X_train.columns if col not in ['building_id',"damage_grade"]]


In [0]:
X_train=X_train[cols]
X_test=X_test[cols]


In [0]:
X = X_train
y = y_train

In [17]:
X_train.columns

Index(['count_families', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'gelogical', 'secondary', 'superstructure',
       'area_assesed_Both', 'area_assesed_Building removed',
       'area_assesed_Exterior', 'area_assesed_Interior',
       'area_assesed_Not able to inspect', 'has_repair_started_0.0',
       'has_repair_started_1.0', 'has_repair_started_-999.0',
       'legal_ownership_status_Institutional', 'legal_ownership_status_Other',
       'legal_ownership_status_Private', 'legal_ownership_status_Public',
       'land_surface_condition_Flat', 'land_surface_condition_Moderate slope',
       'land_surface_condition_Steep slope', 'foundation_type_Bamboo/Timber',
       'foundation_type_Cement-Stone/Brick',
       'foundation_type_Mud mortar-Stone/Brick', 'foundation_type_Other',
       'foundation_type_RC', 'roof_type_Bamboo/Timber-Heavy roof',
       'roof_type_Bamboo/Timber-Light roof', 'r

In [20]:
y

459058    4
119934    4
157320    5
518143    2
156271    4
418291    5
433457    5
170330    5
495003    2
292372    2
599974    2
600797    4
52457     3
425803    5
72659     5
4367      2
291993    4
97954     2
519962    3
479272    3
119585    3
354402    3
42817     3
475928    4
157206    4
236044    4
418709    5
237947    5
405459    3
283819    3
         ..
149503    5
384681    5
156730    3
500186    4
486232    4
258795    5
421909    5
347449    5
184779    4
214176    5
235796    5
103355    3
199041    4
327069    3
321879    5
64820     5
329365    1
41090     2
278167    5
191335    4
175203    5
87498     5
521430    4
137337    1
54886     4
110268    2
259178    5
365838    5
131932    2
121958    2
Name: damage_grade, Length: 410644, dtype: int64

In [0]:
cols = [col for col in X_test.columns if col not in ['building_id',"damage_grade"]]


In [67]:
X_train['count_families']

459058    1.0
119934    1.0
157320    1.0
518143    1.0
156271    1.0
418291    1.0
433457    1.0
170330    1.0
495003    1.0
292372    1.0
599974    1.0
600797    1.0
52457     1.0
425803    1.0
72659     1.0
4367      1.0
291993    1.0
97954     1.0
519962    1.0
479272    0.0
119585    0.0
354402    2.0
42817     1.0
475928    1.0
157206    0.0
236044    1.0
418709    1.0
237947    1.0
405459    1.0
283819    1.0
         ... 
149503    1.0
384681    1.0
156730    1.0
500186    1.0
486232    2.0
258795    1.0
421909    0.0
347449    1.0
184779    1.0
214176    2.0
235796    1.0
103355    1.0
199041    1.0
327069    1.0
321879    1.0
64820     1.0
329365    1.0
41090     1.0
278167    1.0
191335    1.0
175203    1.0
87498     1.0
521430    1.0
137337    1.0
54886     1.0
110268    1.0
259178    1.0
365838    1.0
131932    1.0
121958    1.0
Name: count_families, Length: 410644, dtype: float64

In [0]:
dtrain = xgb.DMatrix(X_train, label=y_train,missing=-999.0)
dtest = xgb.DMatrix(X_test, label=y_test,missing=-999.0)

In [32]:
X_test

,count_families,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,gelogical,secondary,superstructure,...,condition_post_eq_Damaged-Repaired and used,condition_post_eq_Damaged-Rubble Clear-New building built,condition_post_eq_Damaged-Rubble clear,condition_post_eq_Damaged-Rubble unclear,condition_post_eq_Damaged-Used in risk,condition_post_eq_Not damaged,has_geotechnical_risk_0,has_geotechnical_risk_1,has_secondary_use_0,has_secondary_use_1
272472,1.0,2.0,2.0,7.0,468.0,24.0,24.0,0.0,0.0,2.0,...,0,0,0,0,1,0,1,0,1,0
453093,1.0,1.0,1.0,1.0,268.0,9.0,9.0,0.0,0.0,2.0,...,0,0,0,0,0,1,1,0,1,0
83215,1.0,3.0,1.0,28.0,450.0,21.0,7.0,2.0,0.0,1.0,...,0,0,0,0,1,0,0,1,1,0
387516,1.0,1.0,1.0,21.0,225.0,8.0,8.0,0.0,0.0,1.0,...,1,0,0,0,0,0,1,0,1,0
15368,1.0,3.0,3.0,40.0,288.0,10.0,10.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,1,0
1589,1.0,2.0,2.0,15.0,288.0,16.0,16.0,0.0,0.0,1.0,...,1,0,0,0,0,0,1,0,1,0
378796,1.0,3.0,0.0,15.0,884.0,27.0,0.0,0.0,0.0,1.0,...,0,0,1,0,0,0,1,0,1,0
525062,1.0,3.0,0.0,10.0,166.0,12.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
371827,1.0,4.0,0.0,100.0,648.0,28.0,0.0,0.0,0.0,2.0,...,0,0,0,1,0,0,1,0,1,0
146759,1.0,2.0,2.0,30.0,445.0,13.0,13.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,1,0


In [0]:
X=X_train
Y=y_train

In [0]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 0,  # logging mode - quiet
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 6,
    'verbose':True    } 

In [0]:
bst = xgb.train(param, dtrain, 50,verbose_eval=True)

In [0]:
bst

In [0]:
y_pred = bst.predict(dtrain)

In [0]:
y_test_pred = bst.predict(dtest)

In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_train, y_pred)


In [64]:
cm

array([[ 36810,   8815,   3400,    209,      3],
       [  1307,  31113,  24799,  10700,     12],
       [   410,  19974,  40452,  36899,     25],
       [   100,   6276,  23925,  91116,    150],
       [    16,    226,    773,   7441, 160457]])

In [73]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(y_train, y_pred)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [0.98476621 0.47341815 0.42238795 0.61053466 0.99941488]
recall: [0.81449601 0.40488647 0.42138468 0.74652536 0.94844668]
fscore: [0.89157449 0.43647864 0.42188571 0.67171621 0.97326396]
support: [ 40080  55050  79571  99075 136868]


In [96]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_score(y_train, y_pred)
#accuracy_score(y_test, y_test_pred)


0.711657786306387

In [76]:
f1_score(y_train, y_pred, average='weighted')


0.7137357358644119

In [70]:
precision_score(y_train, y_pred, average='micro')  

0.7121929213625428

In [71]:
 from sklearn.metrics import recall_score
 recall_score(y_train, y_pred, average='micro')

0.7121929213625428

In [0]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search


In [0]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [0]:
predictors = [x for x in train.columns if x not in ['building_id','damage_grade']]


In [0]:
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, missing=-999.0,n_estimators=140, max_depth=5,min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=16, scale_pos_weight=1, seed=27), param_grid = params, scoring='f1_micro',n_jobs=5,iid=False, cv=5,verbose=5)


In [113]:
gsearch1.fit(X,Y)
#gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

Fitting 5 folds for each of 405 candidates, totalling 2025 fits
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8 


Process ForkPoolWorker-39:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/pool.py", line 362, in get
    return recv()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/numpy_pickle.py", line 578, in load
    obj = _unpickle(fobj, filename, mmap_mode)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/numpy_pickle.py", line 508, in _unpickle
    obj = unpickler.load()
  File "/usr/lib/python3.6/pickle.py", line 1050, in load
    dispatch[key[0]](self)
  File "/us

KeyboardInterrupt: ignored

In [0]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [0]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softmax',silent=False, nthread=16)

In [0]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_micro', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001)

In [0]:
Y = train_new['damage_grade'].values
X = train_new.drop(['damage_grade'], axis=1)

In [107]:
X

,count_families,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,area_assesed_Both,area_assesed_Building removed,area_assesed_Exterior,...,has_superstructure_other_0,has_superstructure_other_1,condition_post_eq_Covered by landslide,condition_post_eq_Damaged-Not used,condition_post_eq_Damaged-Repaired and used,condition_post_eq_Damaged-Rubble Clear-New building built,condition_post_eq_Damaged-Rubble clear,condition_post_eq_Damaged-Rubble unclear,condition_post_eq_Damaged-Used in risk,condition_post_eq_Not damaged
0,1,2,2,33,312,14,14,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,1,2,2,13,676,17,17,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,1,1,8,406,12,12,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,1,2,2,13,616,15,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,2,2,40,480,15,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0
5,1,2,2,11,459,15,15,1,0,0,...,1,0,0,0,1,0,0,0,0,0
6,1,2,2,50,390,15,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0
7,1,2,2,50,459,15,15,0,0,1,...,1,0,0,0,0,0,0,0,1,0
8,1,2,1,15,416,15,8,1,0,0,...,1,0,0,1,0,0,0,0,0,0
9,1,2,2,15,629,15,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [117]:
# Here we go
#start_time = timer(1000) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
#timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV] subsample=0.8, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=0.8 


Process ForkPoolWorker-51:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/pool.py", line 362, in get
    return recv()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/numpy_pickle.py", line 578, in load
    obj = _unpickle(fobj, filename, mmap_mode)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/numpy_pickle.py", line 508, in _unpickle
    obj = unpickler.load()
  File "/usr/lib/python3.6/pickle.py", line 1050, in load
    dispatch[key[0]](self)
  File "/us

KeyboardInterrupt: ignored

In [100]:
%env JOBLIB_TEMP_FOLDER=/tmp


env: JOBLIB_TEMP_FOLDER=/tmp


In [102]:
y_train

329383    5
218288    4
40161     3
609708    1
353803    4
153324    4
268479    2
331668    4
351569    4
539836    2
362016    5
101786    1
411886    5
280595    5
73000     5
506097    1
347876    4
295961    1
480900    2
619226    5
260460    5
366313    4
250808    5
206215    5
123956    2
262486    3
173756    3
272840    4
137526    2
64995     3
         ..
149503    3
384681    4
156730    4
500186    5
486232    2
258795    5
421909    1
347449    5
184779    3
214176    5
235796    5
103355    3
199041    4
327069    5
321879    4
64820     5
329365    1
41090     2
278167    5
191335    3
175203    5
87498     5
521430    4
137337    1
54886     4
110268    2
259178    5
365838    4
131932    2
121958    2
Name: damage_grade, Length: 505408, dtype: int64

In [0]:
import lightgbm as lgb


In [119]:
!pip install lightgbm

    100% |████████████████████████████████| 737kB 5.2MB/s 


In [0]:
import lightgbm as lgb

In [0]:
d_train = lgb.Dataset(X, label=y)


In [26]:
y

459058    4
119934    4
157320    5
518143    2
156271    4
418291    5
433457    5
170330    5
495003    2
292372    2
599974    2
600797    4
52457     3
425803    5
72659     5
4367      2
291993    4
97954     2
519962    3
479272    3
119585    3
354402    3
42817     3
475928    4
157206    4
236044    4
418709    5
237947    5
405459    3
283819    3
         ..
149503    5
384681    5
156730    3
500186    4
486232    4
258795    5
421909    5
347449    5
184779    4
214176    5
235796    5
103355    3
199041    4
327069    3
321879    5
64820     5
329365    1
41090     2
278167    5
191335    4
175203    5
87498     5
521430    4
137337    1
54886     4
110268    2
259178    5
365838    5
131932    2
121958    2
Name: damage_grade, Length: 410644, dtype: int64

In [123]:
d_train.feature_name

'auto'

In [0]:

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'softmax'
params['metric'] = 'multi_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
params['num_class'] = 6
params['is_unbalance'] = 'TRUE'



In [44]:
params

{'boosting_type': 'gbdt',
 'is_unbalance': 'TRUE',
 'learning_rate': 0.003,
 'max_depth': 10,
 'metric': 'multi_logloss',
 'min_data': 50,
 'num_class': 6,
 'num_leaves': 10,
 'objective': 'softmax',
 'sub_feature': 0.5}

In [0]:
clf = lgb.train(params, d_train, 100)

In [0]:
y_pred=clf.predict(X_test)


In [38]:
y_pred

array([[1.99728966e-16, 1.69920250e-01, 2.01781169e-01, 2.30897955e-01,
        2.28410272e-01, 1.68990354e-01],
       [1.96337359e-16, 3.31147806e-01, 1.68207493e-01, 1.68209008e-01,
        1.66919687e-01, 1.65516007e-01],
       [2.02745395e-16, 1.71829474e-01, 1.92660240e-01, 1.98364075e-01,
        2.55349535e-01, 1.81796676e-01],
       ...,
       [2.00054969e-16, 1.69772541e-01, 2.01677692e-01, 2.31238580e-01,
        2.28045002e-01, 1.69266184e-01],
       [2.02709183e-16, 1.82886144e-01, 2.24629345e-01, 2.16841967e-01,
        1.99769679e-01, 1.75872864e-01],
       [1.98480942e-16, 1.67859266e-01, 1.80904889e-01, 2.20689878e-01,
        2.59696669e-01, 1.70849298e-01]])

In [0]:
predictions = []

for x in y_pred:
    predictions.append(np.argmax(x))

In [41]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_score(y_test, predictions)

0.7022345636020749